# EDA
## A ideia é: EDA gera hipóteses → Modelagem aplica as decisões.

In [ ]:
# === 01 • Imports e carga dos dados tratados ===
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

DATA_PROCESSED = "../data/processed/listings_model_baseline.csv"

df = pd.read_csv(DATA_PROCESSED)
print(f"Shape: {df.shape}")
df.head(3)


In [ ]:
# === 02 • Distribuição dos preços ===
plt.figure(figsize=(10,5))
sns.histplot(df["price"], bins=100, kde=True)
plt.title("Distribuição dos preços das diárias (Airbnb)")
plt.xlabel("Preço")
plt.ylabel("Frequência")
plt.show()


## 🔹 O que o gráfico mostra

No eixo X (horizontal): os valores de preço da diária (em dólares).

No eixo Y (vertical): a frequência de quantos anúncios têm aquele preço.

As barras formam o histograma, e a linha azul é a densidade estimada (KDE), que suaviza a curva da distribuição.

🔹 Interpretação

Distribuição assimétrica (cauda longa à direita)

A maior parte das diárias está concentrada entre 50 e 200 dólares.

Mas há uma cauda longa: ainda existem anúncios com preços muito mais altos (600, 1000, até 1300+).

Moda clara (pico principal)

O pico entre 100–150 dólares mostra onde está a maior frequência de preços anunciados.

Isso indica o “valor típico” praticado pela maioria dos hosts.

Outliers evidentes

Apesar de poucos, existem anúncios com preços fora do padrão (valores acima de 800–1000 dólares).

Esses outliers podem distorcer modelos de regressão se não forem tratados.

🔹 Implicações para o modelo

Transformação logarítmica pode ser útil (log(price+1)) → ajuda a reduzir o impacto da cauda longa e facilita o ajuste de regressão.

Remover ou limitar outliers (winsorizar, por exemplo cortando no percentil 99) → evita que preços absurdos “puxem” o modelo para cima.

Feature engineering: talvez valha calcular métricas baseadas no preço por pessoa (price/accommodates) para normalizar anúncios com capacidades muito diferentes.

In [ ]:
# === 03 • Estatísticas descritivas ===
df["price"].describe(percentiles=[.05,.25,.5,.75,.95])


In [ ]:
# === 04 • Preço por tipo de quarto ===
plt.figure(figsize=(8,5))
sns.boxplot(data=df, x="room_type", y="price")
plt.ylim(0, 500)  # limitar para ver melhor a mediana
plt.title("Distribuição de preços por tipo de quarto")
plt.show()


## 🔹 O que o gráfico mostra

Cada caixa representa a distribuição dos preços para um tipo de quarto.

A linha dentro da caixa = mediana (valor central).

As bordas da caixa = 1º e 3º quartis (50% dos dados estão ali).

Os “bigodes” = valores dentro de 1.5×IQR (intervalo interquartílico).

Pontos fora dos bigodes = outliers.

🔹 Interpretação por categoria

Private room

Mediana em torno de 80–100 dólares.

Grande concentração de valores baixos.

Muitos outliers acima de 200–400 dólares, mas são exceções.

Entire home/apt

Mediana mais alta, em torno de 140–160 dólares.

Caixa mais “esticada”, indicando grande variação de preços.

Muitos outliers, incluindo valores acima de 400 dólares.

É esperado: casas/apartamentos inteiros variam bastante conforme tamanho/localização.

Hotel room

Mediana em torno de 160 dólares, levemente acima de Entire home/apt.

Distribuição menos espalhada, mas ainda com alguns outliers (~400).

Mostra que quartos de hotel tendem a ter preço mais estável que apartamentos.

Shared room

Valores muito mais baixos (mediana ~50 dólares).

Pouca variação → todos estão em uma faixa parecida.

Esse tipo é bem mais acessível e previsível.

🔹 Insights para o modelo

O tipo de quarto é um forte preditor de preço → deve ser mantido como feature categórica no modelo.

Outliers de Entire home/apt e Private room confirmam a necessidade de tratar preços extremos (senão o modelo pode tentar “explicar” valores muito fora da curva).

Talvez valha a pena criar um feature derivado como relative_price = price / median_price_by_room_type, para normalizar diferenças entre categorias.

In [ ]:
# === 05 • Correlação entre variáveis numéricas ===
num_cols = df.select_dtypes(include=["float64","int64"]).columns
plt.figure(figsize=(12,8))
sns.heatmap(df[num_cols].corr(), annot=False, cmap="coolwarm", center=0)
plt.title("Mapa de correlação entre variáveis numéricas")
plt.show()


In [ ]:
# === 06 • Mapa de calor: Preço x Localização ===
plt.figure(figsize=(10,8))
sc = plt.scatter(
    df["longitude"], df["latitude"],
    c=df["price"], cmap="viridis", alpha=0.6, s=10
)
plt.colorbar(sc, label="Preço da diária")
plt.title("Mapa de calor: preço por localização")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


## 🔹 O que o gráfico mostra

Cada célula representa a correlação de Pearson entre duas variáveis.

Valores vão de -1 a 1:

+1 → correlação perfeita positiva (quando uma cresce, a outra também).

-1 → correlação perfeita negativa (quando uma cresce, a outra diminui).

0 → sem correlação linear significativa.

A escala de cores vai do azul (negativa) ao vermelho (positiva).

🔹 Interpretação dos resultados principais

Correlação forte entre variáveis de capacidade

accommodates, bedrooms, bathrooms, beds → apresentam correlações altas (vermelho intenso).

Faz sentido: quanto mais pessoas o imóvel acomoda, mais quartos/camas/banheiros tende a ter.

Isso sugere redundância → incluir todas pode gerar multicolinearidade; talvez seja bom selecionar só algumas.

Preço e capacidade

price tem correlação positiva com accommodates, bedrooms e beds.

Confirma a intuição: imóveis maiores costumam ser mais caros.

Preço e localização

latitude e longitude têm correlação baixa com price.

Isso não significa que localização não importa, mas sim que a relação não é linear (pode ser captada melhor com modelos de árvore).

Preço e reviews

number_of_reviews e reviews_per_month não têm correlação clara com preço (próximas de zero).

Ou seja, a popularidade não define diretamente o valor da diária.

Notas de avaliação

review_scores_rating também não mostra correlação forte com preço → hóspedes avaliam qualidade, mas não necessariamente preço.

Variáveis de noites mínimas/máximas

minimum_nights e maximum_nights não se correlacionam fortemente com preço → hosts podem configurar esses valores de forma arbitrária.

Variáveis binárias (instant_bookable, host_is_superhost)

Correlações fracas com preço.

Isso indica que disponibilidade imediata ou ser superhost não alteram diretamente o preço, mas podem influenciar a procura.

🔹 Implicações para o modelo

Feature selection:

Devemos avaliar se usamos todas as variáveis altamente correlacionadas (bedrooms, beds, accommodates) ou escolhemos apenas algumas para evitar redundância.

Localização:

Embora correlação linear seja baixa, modelos não lineares (como Random Forest ou XGBoost) podem capturar melhor o impacto geográfico.

Foco inicial:

Para o modelo baseline, variáveis de capacidade e tipo de quarto já devem explicar boa parte da variabilidade do preço.